<a href="https://colab.research.google.com/github/Bobain/LBKE_langchain_langgraph/blob/main/notebooks/Copie_de_Formation_LangChain_jour_1_apr%C3%A8s_midi_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction à LangChain - embeddings et RAG
https://www.lbke.fr/formations/developpeur-llm-langgraph-langchain  
https://www.lbke.fr/formations/developpeur-llm-langgraph-langchain/cpf

## Installation

In [ ]:
# La syntaxe "langchain[mistralai]" installe plusieurs dépendances
# Voir la notion d'extras en Python : https://packaging.python.org/en/latest/specifications/dependency-specifiers/#extras
%pip install -qU "langchain[mistralai]"
%pip install -qU langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2

In [ ]:
import os
from google.colab import userdata
os.environ["MISTRAL_API_KEY"] = userdata.get('MISTRAL_API_KEY')

In [ ]:
# We setting LangSmith later in the course but we need this setup to be at top-level
# You can disable langsmith if it creates issues or just ignore this cell
%pip install langsmith
import langsmith as ls
print("Enabling langsmith")
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_API_KEY"]=userdata.get("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"]="formation-intro-langchain"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"


Enabling langsmith


In [ ]:
from langchain.chat_models import init_chat_model
# https://mistral.ai/products/la-plateforme#pricing section "cloud"
model = init_chat_model("codestral-latest", model_provider="mistralai", )
print(type(model)) # LangChain will use the type specific to this provider eg ChatMistralAI

<class 'langchain_mistralai.chat_models.ChatMistralAI'>


In [ ]:
# On vérifie l'accès à Mistral
from langchain_core.messages import HumanMessage
messages = [
    HumanMessage("Génère un court programme Python avec LangChain qui envoie un message à un LLM."),
]
res=model.invoke(messages)

In [ ]:
print(res.content[0:30])

Voici un exemple simple de pro


## Prompt engineering
(repris du premier notebook d'intro)

### Templates

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
system_prompt="""
You are a brilliant Python developer, with a strong knowledge of the most modern syntax of the LangChain framework.
You have a good idea of how LLMs work.
You are going to be asked to generate programs that communicates with LLMs.
Use the LangChain framework. Use the latest syntax with "Chat Models".
Load API keys using Google Colab secrets using "userdata".
Include relevant "%pip install" commands.
Use {model}.
Output the code within XML <program> tags. Output only the code in your response.
"""
prompt_template = ChatPromptTemplate(
    [("system", system_prompt), ("user", "{instruction}")],
    input_variables=["model", "instruction"],
    partial_variables={"model": "Mistral AI"}
)
prompt_template.invoke({"model": "Anthropic", "instruction": "Generate a short Python program that sends a message to an LLM."})

ChatPromptValue(messages=[SystemMessage(content='\nYou are a brilliant Python developer, with a strong knowledge of the most modern syntax of the LangChain framework.\nYou have a good idea of how LLMs work.\nYou are going to be asked to generate programs that communicates with LLMs.\nUse the LangChain framework. Use the latest syntax with "Chat Models".\nLoad API keys using Google Colab secrets using "userdata".\nInclude relevant "%pip install" commands.\nUse Anthropic.\nOutput the code within XML <program> tags. Output only the code in your response.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Generate a short Python program that sends a message to an LLM.', additional_kwargs={}, response_metadata={})])

## Embeddings

In [ ]:
from langchain_mistralai import MistralAIEmbeddings
# /!\ Mistral embedding models doesn't support dimensions with "coarse-to-fine" approach
# as open AI does!
embeddings = MistralAIEmbeddings(
    model="mistral-embed",
    # should match your API limits
    max_concurrent_requests=6
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
embedding=embeddings.embed_query("Generate a short Python program that sends a message to an LLM.")
print(len(embedding))
embedding[0:10]

1024


[-0.024749755859375,
 0.086181640625,
 0.0171966552734375,
 -0.0212249755859375,
 0.055938720703125,
 0.0160675048828125,
 0.016632080078125,
 -0.0102081298828125,
 0.0184783935546875,
 -0.01165008544921875]

In [ ]:
%pip install -qU scipy
from scipy.spatial.distance import cosine
docs_embeddings=embeddings.embed_documents(
    ["Build a simple LLM application with chat models and prompt templates",
     "Retrieval Augmented Generation (RAG) Part 1"])
# Distance cosine faible <=> similarité cosine forte
# => des embeddings sémantiquement proches
print([cosine(embedding,docs_embeddings[0]), cosine(embedding, docs_embeddings[1])])

[np.float64(0.16043110301505925), np.float64(0.27468118704801237)]


In [ ]:
len(docs_embeddings)

2

## Ingérer des pages web

In [ ]:
url_to_parse="https://docs.langchain.com/oss/python/langchain/overview" # new agent API from langchain v1 alpha

### Avec BeautifulSoup

In [ ]:
%pip install -qU langchain-community beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 3.0 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import bs4 # beautifulsoup4
loader = WebBaseLoader(url_to_parse,
                       # pour extraire le tag "article" uniquemen
                       bs_kwargs={
        "parse_only": bs4.SoupStrainer("article"),
    })
docs = loader.load()
doc = docs[0]
print(f"# {doc.metadata}\n\n")
print(doc.page_content[0:5000])

# {'source': 'https://docs.langchain.com/oss/python/langchain/overview'}





In [ ]:
print(docs)

[Document(metadata={'source': 'https://docs.langchain.com/oss/python/langchain/overview'}, page_content='')]


### Avec docling

In [ ]:
%pip install -qU langchain-docling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.3/169.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 33.8 MB/s eta 0:00:00
   

In [ ]:
# Exemple de chargement d'un document
from langchain_docling.loader import ExportType, DoclingLoader
loader = DoclingLoader(url_to_parse, export_type=ExportType.MARKDOWN)
docs = loader.load()
doc=docs[0] # If we don't use "markdown", we will get multiple document chunks instead
print(len(docs))
from IPython.display import Markdown
Markdown("\n\n".join([doc.page_content for doc in docs]))

1


##### LangChain v1.0

- [Release notes](/oss/python/releases/langchain-v1)
- [Migration guide](/oss/python/migrate/langchain-v1)

##### Get started

- [Install](/oss/python/langchain/install)
- [Quickstart](/oss/python/langchain/quickstart)
- [Philosophy](/oss/python/langchain/philosophy)

##### Core components

- [Agents](/oss/python/langchain/agents)
- [Models](/oss/python/langchain/models)
- [Messages](/oss/python/langchain/messages)
- [Tools](/oss/python/langchain/tools)
- [Short-term memory](/oss/python/langchain/short-term-memory)
- [Streaming](/oss/python/langchain/streaming)
- [Middleware](/oss/python/langchain/middleware)
- [Structured output](/oss/python/langchain/structured-output)

##### Advanced usage

- [Guardrails](/oss/python/langchain/guardrails)
- [Runtime](/oss/python/langchain/runtime)
- [Context engineering](/oss/python/langchain/context-engineering)
- [Model Context Protocol (MCP)](/oss/python/langchain/mcp)
- [Human-in-the-loop](/oss/python/langchain/human-in-the-loop)
- [Multi-agent](/oss/python/langchain/multi-agent)
- [Retrieval](/oss/python/langchain/retrieval)
- [Long-term memory](/oss/python/langchain/long-term-memory)

##### Use in production

- [Studio](/oss/python/langchain/studio)
- [Test](/oss/python/langchain/test)
- [Deploy](/oss/python/langchain/deploy)
- [Agent Chat UI](/oss/python/langchain/ui)
- [Observability](/oss/python/langchain/observability)

On this page

- [Install](#install)
- [Create an agent](#create-an-agent)
- [Core benefits](#core-benefits)

# LangChain overview

Copy page

Copy page

**LangChain v1.0 is now available!** For a complete list of changes and instructions on how to upgrade your code, see the [release notes](/oss/python/releases/langchain-v1) and [migration guide](/oss/python/migrate/langchain-v1) . If you encounter any issues or have feedback, please [open an issue](https://github.com/langchain-ai/docs/issues/new?template=01-langchain.yml) so we can improve. To view v0.x documentation, [go to the archived content](https://github.com/langchain-ai/langchain/tree/v0.3/docs/docs) .

LangChain is the easiest way to start building agents and applications powered by LLMs. With under 10 lines of code, you can connect to OpenAI, Anthropic, Google, and [more](/oss/python/integrations/providers/overview) . LangChain provides a pre-built agent architecture and model integrations to help you get started quickly and seamlessly incorporate LLMs into your agents and applications.

We recommend you use LangChain if you want to quickly build agents and autonomous applications. Use [LangGraph](/oss/python/langgraph/overview) , our low-level agent orchestration framework and runtime, when you have more advanced needs that require a combination of deterministic and agentic workflows, heavy customization, and carefully controlled latency.

LangChain [agents](/oss/python/langchain/agents) are built on top of LangGraph in order to provide durable execution, streaming, human-in-the-loop, persistence, and more. You do not need to know LangGraph for basic LangChain agent usage.

## [ Install](#install)

pip uv Copy Ask AI

```
pip install -U langchain
```

## [ Create an agent](#create-an-agent)

Copy Ask AI

```
# pip install -qU "langchain[anthropic]" to call the model

from langchain.agents import create_agent

def get_weather ( city : str ) -> str :
"""Get weather for a given city."""
return f "It's always sunny in { city } !"

agent = create_agent(
model = "claude-sonnet-4-5-20250929" ,
tools = [get_weather],
system_prompt = "You are a helpful assistant" ,
)

# Run the agent
agent.invoke(
{ "messages" : [{ "role" : "user" , "content" : "what is the weather in sf" }]}
)
```

## [ Core benefits](#core-benefits)

## [Standard model interface](/oss/python/langchain/models)

[Different providers have unique APIs for interacting with models, including the format of responses. LangChain standardizes how you interact with models so that you can seamlessly swap providers and avoid lock-in. Learn more](/oss/python/langchain/models)

## [Easy to use, highly flexible agent](/oss/python/langchain/agents)

[LangChain's agent abstraction is designed to be easy to get started with, letting you build a simple agent in under 10 lines of code. But it also provides enough flexibility to allow you to do all the context engineering your heart desires. Learn more](/oss/python/langchain/agents)

## [Built on top of LangGraph](/oss/python/langgraph/overview)

[LangChain's agents are built on top of LangGraph. This allows us to take advantage of LangGraph's durable execution, human-in-the-loop support, persistence, and more. Learn more](/oss/python/langgraph/overview)

## [Debug with LangSmith](/langsmith/home)

[Gain deep visibility into complex agent behavior with visualization tools that trace execution paths, capture state transitions, and provide detailed runtime metrics. Learn more](/langsmith/home)

[Edit the source of this page on GitHub.](https://github.com/langchain-ai/docs/edit/main/src/oss/langchain/overview.mdx)

[Connect these docs programmatically](/use-these-docs) to Claude, VSCode, and more via MCP for    real-time answers.

Was this page helpful?

Yes No

[What's new in v1 Next](/oss/python/releases/langchain-v1) ⌘ I

### Découper des documents

In [ ]:
# Configuration du splitter
from langchain_text_splitters import MarkdownHeaderTextSplitter
# /!\ MarkdownHeaderTextSplitter ne limite pas la longueur,
# TODO: il faudrait le coupler avec un Recursive pour limiter la taille des blocs
splitter = MarkdownHeaderTextSplitter([("#", "title 1"), ("##", "title 2"), ("###", "title 3")])

In [ ]:
# Exemple de splitting d'un document
chunks=splitter.split_text(doc.page_content)
print(len(chunks), [len(chunk.page_content) for chunk in chunks])

8 [1529, 1535, 53, 491, 256, 283, 211, 537]


In [ ]:
# Pour le debug
from langchain_core.documents import Document
def display_chunk(chunk:Document)->str:
  return f"""
  # {chunk.metadata.get('title 1')}
  ## {chunk.metadata.get('title 2')}
  ### {chunk.metadata.get('title 3')}
  {chunk.page_content}
  """
print("\n\n---\n\n".join([display_chunk(chunk) for chunk in chunks]))


  # None
  ## None
  ### None
  ##### LangChain v1.0  
- [Release notes](/oss/python/releases/langchain-v1)
- [Migration guide](/oss/python/migrate/langchain-v1)  
##### Get started  
- [Install](/oss/python/langchain/install)
- [Quickstart](/oss/python/langchain/quickstart)
- [Philosophy](/oss/python/langchain/philosophy)  
##### Core components  
- [Agents](/oss/python/langchain/agents)
- [Models](/oss/python/langchain/models)
- [Messages](/oss/python/langchain/messages)
- [Tools](/oss/python/langchain/tools)
- [Short-term memory](/oss/python/langchain/short-term-memory)
- [Streaming](/oss/python/langchain/streaming)
- [Middleware](/oss/python/langchain/middleware)
- [Structured output](/oss/python/langchain/structured-output)  
##### Advanced usage  
- [Guardrails](/oss/python/langchain/guardrails)
- [Runtime](/oss/python/langchain/runtime)
- [Context engineering](/oss/python/langchain/context-engineering)
- [Model Context Protocol (MCP)](/oss/python/langchain/mcp)
- [Human-in-the-

### Fonction d'ingestion des URLs par lots

In [ ]:
# Code final en combinant toute la logique
from langchain_docling.loader import ExportType, DoclingLoader
from typing import List
from langchain_core.documents import Document

# NOTE: we could use a SitemapLoader instead
# https://python.langchain.com/docs/integrations/document_loaders/sitemap/
def load_urls(urls: List[str])->List[Document]:
  chunks=[]
  batch_size=4
  batches=len(urls)//batch_size
  for batch_idx in range(batches+1):
    print(f"Retrieving batch {batch_idx+1} over {batches+1}")
    urls_batch = urls[batch_idx*batch_size:(batch_idx+1)*batch_size]
    #print(urls_batch)
    loader = DoclingLoader(urls_batch, export_type=ExportType.MARKDOWN)
    docs = loader.load()
    # Les fonctions du splitter peuvent varier selon le type
    # Recursive possède une fonction "split_documents"
    # MarkdownTextHeader n'en possède pas, mais son "split_text" renvoie des documents
    for doc in docs:
      splitted_docs = splitter.split_text(doc.page_content)
      chunks = chunks + splitted_docs
  return chunks

In [ ]:
# Test de la fonction
langchain_docs_urls=[
    "https://docs.langchain.com/oss/python/langchain/install",
    "https://docs.langchain.com/oss/python/langchain/quickstart",
    "https://docs.langchain.com/oss/python/langchain/agents"
]
len(load_urls(langchain_docs_urls))

Retrieving batch 1 over 1


16

In [ ]:
# Bonus: SitemapLoader
# https://python.langchain.com/docs/integrations/document_loaders/sitemap/
# Attention aux attaques SSRF = urls localhost dans une sitemap
# import nest_asyncio
# nest_asyncio.apply()
# sitemap_url="https://python.langchain.com/sitemap.xml"
# from langchain_community.document_loaders import SitemapLoader
# sitemap_loader=SitemapLoader(sitemap_url)
# sitemap_docs=sitemap_loader.load()

In [ ]:
sitemap_chunks=[]
for doc in sitemap_docs:
  splitted_docs = splitter.split_text(doc.page_content)
  sitemap_chunks = sitemap_chunks + splitted_docs
sitemap_docs[0]

NameError: name 'sitemap_docs' is not defined

## Vector search

In [ ]:
from langchain_mistralai import MistralAIEmbeddings
# /!\ Mistral embedding models doesn't support dimensions with "coarse-to-fine" approach
# as open AI does!
embeddings = MistralAIEmbeddings(
    model="mistral-embed",
    # should match your API limits
    max_concurrent_requests=6
)

In [ ]:
# Open AI alternative
# %pip install -qU langchain_openai
# from langchain_openai import OpenAIEmbeddings
# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# embeddings= OpenAIEmbeddings()

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

In [ ]:
def ingest_urls(urls:List[str]):
  docs = load_urls(urls)
  # Mistral has batching out of the box, we just need to properly configure the embedding model
  # /!\ debugging capabilities are limited as we can't observe mistral HTTP requests from LangSmith
  # A single failure will fail the whole batch of documents
  vector_store.add_documents(docs)

In [ ]:
# In case of errors, we can debug with logging httpx (used by mistral embedding model) https://www.python-httpx.org/logging/
# see https://github.com/langchain-ai/langchain/issues/30524
# we can use openai as a fallback
ingest_urls(langchain_docs_urls[0:10])

Retrieving batch 1 over 1


In [ ]:
# sitemap version
# vector_store.add_documents(sitemap_chunks)

In [ ]:
results = vector_store.similarity_search("Generate a short Python program that sends a message to an LLM. Use LangChain v1 agent syntax.")

In [ ]:
len(results)

4

In [ ]:
print('\n'.join([str(result.metadata) for result in results]))

{}
{}
{'title 1': 'Quickstart', 'title 2': '[ Build a basic agent](#build-a-basic-agent)'}
{}


## Un RAG



In [ ]:
# La recherche de document à partir de la requête =>
# Première étape de la nouvelle chaîne
def augment_query(query):
    closest_documents=vector_store.as_retriever(search_kwargs={"k": 4}).get_relevant_documents(query)
    # Fusion des chaînes de caractère
    documents = '\n---\n\n'.join([doc.page_content for doc in closest_documents])
    return {
    "instruction":query,
    "documents": documents
    }

In [ ]:
print(augment_query("Generate a short Python program that sends a message to an LLM. Use LangChain v1 agent syntax.")["documents"])

/tmp/ipython-input-1339105435.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use `invoke` instead.
  closest_documents=vector_store.as_retriever(search_kwargs={"k": 4}).get_relevant_documents(query)


##### LangChain v1.0  
- [Release notes](/oss/python/releases/langchain-v1)
- [Migration guide](/oss/python/migrate/langchain-v1)  
##### Get started  
- [Install](/oss/python/langchain/install)
- [Quickstart](/oss/python/langchain/quickstart)
- [Philosophy](/oss/python/langchain/philosophy)  
##### Core components  
- [Agents](/oss/python/langchain/agents)
- [Models](/oss/python/langchain/models)
- [Messages](/oss/python/langchain/messages)
- [Tools](/oss/python/langchain/tools)
- [Short-term memory](/oss/python/langchain/short-term-memory)
- [Streaming](/oss/python/langchain/streaming)
- [Middleware](/oss/python/langchain/middleware)
- [Structured output](/oss/python/langchain/structured-output)  
##### Advanced usage  
- [Guardrails](/oss/python/langchain/guardrails)
- [Runtime](/oss/python/langchain/runtime)
- [Context engineering](/oss/python/langchain/context-engineering)
- [Model Context Protocol (MCP)](/oss/python/langchain/mcp)
- [Human-in-the-loop](/oss/python/langchain/human

In [ ]:
# Le template de prompt avec les documents
from langchain_core.prompts import ChatPromptTemplate
system_prompt="""
You are a brilliant Python developer, with a strong knowledge of the most modern syntax of the LangChain framework.
You have a good idea of how LLMs work.
You are going to be asked to generate programs that communicates with LLMs.
Use the LangChain framework. Use the latest syntax with "Chat Models".
Load API keys using Google Colab secrets using "userdata".
Google Colab secret names are strictly identic to the environment variable name, including casing, for example "MISTRAL_API_KEY".
Include relevant "%pip install -qU" commands.
Use {model}.
"""
user_prompt = """
{instruction}

Here are documentation elements that can help you:
{documents}
"""
prompt_template = ChatPromptTemplate(
    [("system", system_prompt), ("user", user_prompt)],
    input_variables=["model", "instruction", "documents"],
    partial_variables={"model": "Mistral AI"}
)

In [ ]:
from typing import TypedDict
class ProgramSchema(TypedDict):
    program: str
model_with_output=model.with_structured_output(ProgramSchema)

In [ ]:
chain = augment_query | prompt_template | model_with_output | (lambda x: x["program"])

In [ ]:
res = chain.invoke(""""
Generate a short Python program that sends a message to an LLM.
Use LangChain v1 agent syntax.
""")
print(res)

import os
from getpass import getpass

from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_openai import ChatOpenAI

# Load API keys using Google Colab secrets
os.environ["MISTRAL_API_KEY"] = getpass("Enter your Mistral API key: ")

# Define a simple tool
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

# Convert the tool to an OpenAI tool
tools = [convert_to_openai_tool(multiply)]

# Create a prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# Initialize the ChatOpenAI model
llm = ChatOpenAI(model="mistral-large-latest", temperature=0)

# Bind the tools to the model
llm_with_tools = llm.bind(tools=tools)

# Create a c

In [ ]:
# Variante bonus avec des runnables
# La recherche (version bonus avec un runnable/via le langage LCEL)
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
augment_query_runnable = RunnableParallel({
    "instruction": RunnablePassthrough(),
    "documents": vector_store.as_retriever(search_kwargs={"k": 4}) | (lambda docs: '\n\n'.join([doc.page_content for doc in docs]))
    })
chain_runnable = augment_query_runnable | prompt_template | model_with_output | (lambda x: x["program"])
res_runnable = chain_runnable.invoke("Generate a short Python program that sends a message to an LLM. Use LangChain alpha v1 agent syntax")
print(res_runnable)

from google.colab import userdata

%pip install -qU langchain-mistral

from langchain.agents import create_agent
from langchain_mistral import ChatMistral

mistral_api_key = userdata.get('MISTRAL_API_KEY')

model = ChatMistral(api_key=mistral_api_key, model="mistral-small-latest")
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city} !"

agent = create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)
